<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train2_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train2 = pd.read_csv(r'/content/train_data2.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train2['Target'] = train2['Target'].replace(['Others'],'Invalid')


In [ ]:
train2['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train2_Xtfidf = tfidf_vect.fit_transform(train2['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train2_Xtfidf,train2['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6559734513274337
                  precision    recall  f1-score   support

        Analysis       0.48      0.32      0.38       126
      Conclusion       0.75      0.47      0.58        45
           Facts       0.73      0.87      0.79       480
         Invalid       0.48      0.57      0.52       132
           Issue       0.59      0.45      0.51        51
Rule/Law/Holding       0.72      0.26      0.38        70

        accuracy                           0.66       904
       macro avg       0.62      0.49      0.53       904
    weighted avg       0.65      0.66      0.64       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 3, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 3, 2, 3, 2, 2, 2, 2, 2, 3, 5, 2, 2, 2, 3, 2, 2]
[0.9976192654941368, 0.9967783946721962, 0.9987650749578517, 0.9986852793927125, 0.99824510297287, 0.9864126550639655, 0.9955397360261665, 0.9906883487311688, 0.9903594440415222, 0.9963288696246062, 0.9977884355596618, 0.9966571034599029, 0.9924679375007983, 0.9978903576777991, 0.9982515777880808, 0.9978104265709066, 0.9956138500759374, 0.9985814226501849, 0.9950289289755521, 0.9970385554595347, 0.9980454399718244, 0.9991593007199359, 0.9803471691483968, 0.998309074407492, 0.9972800566966179, 0.9955568876102349, 0.9939183010801993, 0.9848367982481598, 0.9913813823259013, 0.9940087201383031, 0.994663328958525, 0.9869606676052917, 0.9977901607085391, 0.9967916990892535, 0.9930206312336249, 0.9979467835336814, 0.9974374505494819, 0.9854148992942383, 0.9869452924809711, 0.9985608282164745, 0.9931742375974588, 0.998

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
8,944,relies upon tonnahill v,Invalid


In [ ]:
frame_1 = [train2,unlabel_1]
train2_1 = pd.concat(frame_1)
len(train2_1)

3535

In [ ]:
x_train_1 = tfidf_vect.transform(train2_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train2_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.620933521923621
                  precision    recall  f1-score   support

        Analysis       0.36      0.33      0.35        96
      Conclusion       0.71      0.42      0.53        36
           Facts       0.70      0.84      0.77       367
         Invalid       0.50      0.47      0.48       106
           Issue       0.67      0.35      0.46        52
Rule/Law/Holding       0.54      0.28      0.37        50

        accuracy                           0.62       707
       macro avg       0.58      0.45      0.49       707
    weighted avg       0.61      0.62      0.60       707



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 3, 2, 3, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 2, 5, 2, 2, 5, 3, 5, 2]
[0.9979229663187371, 0.9987573879305799, 0.9850037078726408, 0.9978491609899864, 0.9984628497488395, 0.9964134345440799, 0.9986661776432539, 0.9948579209151766, 0.9940267377762396, 0.9823075851940046, 0.9954391093700858, 0.9928659103179116, 0.9910857093290787, 0.9962910589416011, 0.9967417548110856, 0.9903738219063132, 0.9944529652552643, 0.9985716705364093, 0.9939753686224062, 0.9985700537261255, 0.9972232166751068, 0.9987470643796791, 0.9964472825002091, 0.9941377273737438, 0.9947556774569857, 0.994626195530109, 0.9870787898004695, 0.998342632927656, 0.9922996002023121, 0.9985144434241175, 0.998686192731328, 0.9979389227443977, 0.998677252071008, 0.9981800396974332, 0.9810376732685494, 0.986805507631367, 0.9950709130075244, 0.998351960726641, 0.9933829275859154, 0.9980565565231279, 0.9807721387824146, 0.9950885153010205, 0.996293641996

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [ ]:
frame_2 = [train2_1,unlabel_2]
train2_2 = pd.concat(frame_2)
len(train2_2)

3585

In [ ]:
x_train_2 = tfidf_vect.transform(train2_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train2_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6291322314049587
                  precision    recall  f1-score   support

        Analysis       0.48      0.36      0.41       150
      Conclusion       0.59      0.37      0.45        46
           Facts       0.66      0.88      0.76       473
         Invalid       0.58      0.50      0.54       166
           Issue       0.74      0.34      0.47        67
Rule/Law/Holding       0.60      0.27      0.37        66

        accuracy                           0.63       968
       macro avg       0.61      0.45      0.50       968
    weighted avg       0.62      0.63      0.60       968



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2, 2, 4, 2, 2, 2, 2]
[0.9927967769118142, 0.9885160010772184, 0.9961811538581603, 0.9979658061764902, 0.9967460822955203, 0.9982376670108615, 0.998279575784932, 0.9915108054028658, 0.9822631236073011, 0.9983563549432989, 0.9845498109474573, 0.9984395137407137, 0.9981479046700557, 0.9979523928505836, 0.9955232842318323, 0.998089533114765, 0.9863776017802572, 0.997169996864655, 0.9975768818608354, 0.998167073280374, 0.9908597768109545, 0.9982372954157614, 0.985777926639553, 0.991250951266642, 0.9981660072767912, 0.9972949009020068, 0.9982004567499794, 0.9921713870147494, 0.9964139919181656, 0.9978063541271681, 0.9975463512683223, 0.9852807213859619, 0.9982372954157614, 0.9959747172001158, 0.9959540454965997, 0.9976868549016857, 0.9976158149134061, 0.9977912201105732, 0.9982094968809788, 0.9966756913304127, 0.9810518111

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train2_2,unlabel_3]
train2_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train2_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train2_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6397260273972603
                  precision    recall  f1-score   support

        Analysis       0.41      0.32      0.36       106
      Conclusion       0.70      0.37      0.48        38
           Facts       0.68      0.89      0.77       366
         Invalid       0.58      0.50      0.54       118
           Issue       0.71      0.37      0.49        46
Rule/Law/Holding       0.68      0.34      0.45        56

        accuracy                           0.64       730
       macro avg       0.63      0.46      0.52       730
    weighted avg       0.63      0.64      0.62       730



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 3, 2, 5, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 2, 2, 2, 5, 3, 0, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.998098702075972, 0.9801734387450699, 0.9974993950794756, 0.9895104779613283, 0.9986392302333487, 0.9964479670164551, 0.9960204331454172, 0.9988519244321562, 0.9985982726832716, 0.9848948741021278, 0.9984329732914609, 0.9974708674378957, 0.9956229410838733, 0.9987352112104947, 0.9983491032840761, 0.9972974550484796, 0.9983201732790473, 0.9902655965935636, 0.9886787629752494, 0.9978521685680541, 0.9981753412063783, 0.998435327776632, 0.9984121984841262, 0.9988673039930385, 0.99809520077591, 0.9986307051045531, 0.9984071579219387, 0.9976127953855563, 0.9898564123464498, 0.9978753843239686, 0.9977088674747012, 0.995302849641487, 0.9928527944103245, 0.9955988437271536, 0.9966900100560725, 0.9905178251059084, 0.9890984384264258, 0.9977158737863202, 0.9969227068867149, 0.9906192259501703, 0.9928021616425698, 0.9

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train2_3,unlabel_4]
train2_4 = pd.concat(frame_4)
len(train2_4)
x_train_4 = tfidf_vect.transform(train2_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train2_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6626746506986028
                  precision    recall  f1-score   support

        Analysis       0.44      0.37      0.40       139
      Conclusion       0.85      0.41      0.55        56
           Facts       0.71      0.90      0.79       524
         Invalid       0.60      0.51      0.55       160
           Issue       0.71      0.41      0.52        54
Rule/Law/Holding       0.52      0.22      0.31        69

        accuracy                           0.66      1002
       macro avg       0.64      0.47      0.52      1002
    weighted avg       0.65      0.66      0.64      1002



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 3, 2, 3, 4, 2, 5, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 3, 0, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 1, 2, 2, 3, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 3, 2]
[0.9881151131603219, 0.9988000278847727, 0.9962481296262174, 0.9961196346947966, 0.9985187815683182, 0.9936013981665034, 0.9982723914446374, 0.9920825018684958, 0.9985413502460146, 0.9948058966265337, 0.9980482900438404, 0.9983691367987054, 0.9807200056603468, 0.9946744977207413, 0.9931451362097077, 0.988363824368098, 0.9950402325786997, 0.9898742302386071, 0.9967018455910974, 0.9987015348721152, 0.9938568851640465, 0.9925516074692475, 0.9965728233107672, 0.9954278552882345, 0.9862015922228683, 0.990303591829496, 0.9856785434265287, 0.980152968938402, 0.9963501449649319, 0.9987449845275119, 0.9986300319320383, 0.9975278228401459, 0.9962481296262174, 0.9979518458916148, 0.9945298738016176, 0.9888064891779116, 0.9960824313629247, 0.9981719149927711, 0.9943778206554715, 0.9967018455910974, 0.9975997745433

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train2_4,unlabel_5]
train2_5 = pd.concat(frame_5)
len(train2_5)
x_train_5 = tfidf_vect.transform(train2_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train2_5['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6622516556291391
                  precision    recall  f1-score   support

        Analysis       0.57      0.42      0.48       112
      Conclusion       0.64      0.44      0.52        36
           Facts       0.71      0.90      0.79       386
         Invalid       0.56      0.52      0.54       109
           Issue       0.67      0.37      0.47        49
Rule/Law/Holding       0.53      0.25      0.34        63

        accuracy                           0.66       755
       macro avg       0.61      0.48      0.53       755
    weighted avg       0.65      0.66      0.64       755



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.744721689059501


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7614709435745577, 0.6521846489157902, 0.6911150135915709, None)
